### PERSIAPAN DATA

In [1]:
import pandas as pd

In [2]:
# Baca data dari file CSV
file_path = 'final_dataset.csv'
df = pd.read_csv(file_path)
df.head()

,Participant_ID,Gender,cuisine,ethnic_food,fav_cuisine,fruit_day,greek_food,indian_food,italian_food,marital_status,persian_food,thai_food,veggies_day,weight,Age,Nationality,Food,Dessert,Drink
0,FPS001,Female,Korean/Asian,5,italian,5,5,5,5,In a relationship,5,5,5,I'm not answering this.,16,Indian,"cereal, frozen yogurt, pizza, fast food, mac a...",Yes,"espresso shown, orange juice, Fresh Juice, Fre..."
1,FPS002,Female,other,4,African,2,2,5,5,In a relationship,5,4,1,190,16,Indian,"cereal, Candy, brownies and soda., Fries, plai...",Yes,"espresso shown, soda, Fresh Juice, Fresh Juice"
2,FPS003,Female,American,2,Anything american style.,5,3,1,3,Single,1,1,4,137,16,Indian,"cereal, Ice cream, cheeseburgers, chips., chic...",Yes,"creamy frapuccino, soda, Fresh Juice, Fresh Juice"
3,FPS004,Female,American,5,Orange chicken and chow mani noodles,5,1,1,5,Single,2,2,5,116,16,Indian,"cereal, Pasta, grandma homemade chocolate cake...",Yes,"espresso shown, soda, Fresh Juice, Fresh Juice"
4,FPS005,Female,American,5,italian,5,4,3,5,In a relationship,3,5,5,123,16,Indian,"cereal, ice cream, cake, chocolate, steak, las...",Yes,"espresso shown, soda, Fresh Juice, Fresh Juice"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Participant_ID  149 non-null    object
 1   Gender          149 non-null    object
 2   cuisine         149 non-null    object
 3   ethnic_food     149 non-null    int64 
 4   fav_cuisine     149 non-null    object
 5   fruit_day       149 non-null    int64 
 6   greek_food      149 non-null    int64 
 7   indian_food     149 non-null    int64 
 8   italian_food    149 non-null    int64 
 9   marital_status  149 non-null    object
 10  persian_food    149 non-null    int64 
 11  thai_food       149 non-null    int64 
 12  veggies_day     149 non-null    int64 
 13  weight          149 non-null    object
 14  Age             149 non-null    int64 
 15  Nationality     149 non-null    object
 16  Food            149 non-null    object
 17  Dessert         149 non-null    object
 18  Drink     

In [5]:
df.isnull().sum()

Participant_ID    0
Gender            0
cuisine           0
ethnic_food       0
fav_cuisine       0
fruit_day         0
greek_food        0
indian_food       0
italian_food      0
marital_status    0
persian_food      0
thai_food         0
veggies_day       0
weight            0
Age               0
Nationality       0
Food              0
Dessert           0
Drink             0
dtype: int64

In [6]:
df.duplicated().sum()

0

### PEMROSESAN DATA

In [7]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [8]:
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['cuisine'] = label_encoder.fit_transform(df['cuisine'])
df['fav_cuisine'] = label_encoder.fit_transform(df['fav_cuisine'])
df['marital_status'] = label_encoder.fit_transform(df['marital_status'])
df['Nationality'] = label_encoder.fit_transform(df['Nationality'])
df['Food'] = label_encoder.fit_transform(df['Food'])
df['Dessert'] = label_encoder.fit_transform(df['Dessert'])
df['Drink'] = label_encoder.fit_transform(df['Drink'])

In [9]:
# Mengonversi kolom 'Participant_ID' menjadi numerik menggunakan LabelEncoder
label_encoder_participant = LabelEncoder()
df['Participant_ID'] = label_encoder_participant.fit_transform(df['Participant_ID'])

In [10]:
# Membagi data menjadi data latih dan data uji
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

### Pembuatan dan Pelatihan Model

In [11]:
# Membangun model collaborative filtering dengan TensorFlow
embedding_size = 10

In [12]:
user_input = tf.keras.layers.Input(shape=(1,), name='user_input')
food_input = tf.keras.layers.Input(shape=(1,), name='food_input')

In [13]:
user_embedding = tf.keras.layers.Embedding(input_dim=df['Participant_ID'].nunique(), output_dim=embedding_size, input_length=1)(user_input)
food_embedding = tf.keras.layers.Embedding(input_dim=df['Food'].nunique(), output_dim=embedding_size, input_length=1)(food_input)

In [14]:
user_vecs = tf.keras.layers.Flatten()(user_embedding)
food_vecs = tf.keras.layers.Flatten()(food_embedding)

In [15]:
prod = tf.keras.layers.Dot(axes=1)([user_vecs, food_vecs])

In [16]:
model = tf.keras.models.Model(inputs=[user_input, food_input], outputs=prod)
model.compile(optimizer='adam', loss='mean_squared_error')

In [17]:
# Melatih model
model.fit([train_data['Participant_ID'], train_data['cuisine']], train_data['Dessert'], epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 1s 161ms/step - loss: 2.3796 - val_loss: 1.8730
Epoch 2/10
3/3 [==============================] - 0s 32ms/step - loss: 2.3780 - val_loss: 1.8730
Epoch 3/10
3/3 [==============================] - 0s 33ms/step - loss: 2.3764 - val_loss: 1.8729
Epoch 4/10
3/3 [==============================] - 0s 36ms/step - loss: 2.3748 - val_loss: 1.8728
Epoch 5/10
3/3 [==============================] - 0s 23ms/step - loss: 2.3730 - val_loss: 1.8726
Epoch 6/10
3/3 [==============================] - 0s 23ms/step - loss: 2.3711 - val_loss: 1.8725
Epoch 7/10
3/3 [==============================] - 0s 25ms/step - loss: 2.3690 - val_loss: 1.8723
Epoch 8/10
3/3 [==============================] - 0s 27ms/step - loss: 2.3666 - val_loss: 1.8720
Epoch 9/10
3/3 [==============================] - 0s 24ms/step - loss: 2.3638 - val_loss: 1.8718
Epoch 10/10
3/3 [==============================] - 0s 24ms/step - loss: 2.3609 - val_loss: 1.8716


###EVALUASI MODEL

In [18]:
# Evaluasi model
test_loss = model.evaluate([test_data['Participant_ID'], test_data['cuisine']], test_data['Dessert'])
print(f'Test Loss: {test_loss}')

1/1 [==============================] - 0s 28ms/step - loss: 2.1965
Test Loss: 2.1964871883392334
